In [3]:
#Importar librerias
import cv2
import os
import pytesseract
import tkinter as tk
from tkinter import ttk, PhotoImage, messagebox
import json
from dotenv import load_dotenv
import serial
import time
import threading
import requests
from screeninfo import get_monitors
load_dotenv()
ip_aws =  os.getenv("IP_BACKEND")

# Ruta al modelo de clasificador en cascada
cascade_model_path = 'cascade.xml'
plate_cascade = cv2.CascadeClassifier(cascade_model_path)

class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.master.configure(bg="#FFFFFF")  # Fondo blanco
        self.master.grid_rowconfigure(0, weight=1)
        self.master.grid_columnconfigure(0, weight=1)
        self.load_image()  # Cargar la imagen
        self.create_widgets()
        self.grid(padx=10, pady=10)  # Añadir relleno alrededor del frame

    def load_image(self):
        # Cargar y mostrar la imagen
        image_path = 'logoOxxoj.png'  # Ruta de la imagen (reemplazar por la ruta adecuada)
        photo = PhotoImage(file=image_path)
        image_label = tk.Label(self.master, image=photo, bg="#FFFFFF")
        image_label.image = photo  # Mantener una referencia para evitar la recolección de basura
        image_label.grid(pady=20)  # Ajustar el relleno según sea necesario

    def create_widgets(self):
        # Nuevo frame para contener los widgets
        self.master = ttk.Frame(self)
        self.master.grid(row=0, column=0, sticky='NSEW')

        # Configurar la cuadrícula para centrar el frame
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)

        #Definir entradas y etiquetas
        self.placa_entry = ttk.Entry(self.master)
        self.placa_entry.grid(row=0, column=1, sticky='W', padx=5, pady=2)

        self.carga_entry = ttk.Entry(self.master)
        self.carga_entry.grid(row=2, column=1, sticky='W', padx=5, pady=2)
        self.monto_entry = ttk.Entry(self.master)
        self.monto_entry.grid(row=4, column=1, sticky='W', padx=5, pady=2)
        self.modelo_entry = ttk.Entry(self.master)
        self.modelo_entry.grid(row=5, column=1, sticky='W', padx=5, pady=2)
        
        self.tipo_entry = ttk.Entry(self.master)
        self.tipo_entry.grid(row=3, column=1, sticky='W', padx=5, pady=2)
        
        self.pedido_entry = ttk.Entry(self.master)
        self.pedido_entry.grid(row=6, column=1, sticky='W', padx=5, pady=2)

        # Placa Entry and Label
        self.placa_label = ttk.Label(self.master, text="Placa")
        self.placa_label.grid(row=0, column=0, sticky='W', padx=5, pady=2)
        
        self.modelo_label = ttk.Label(self.master, text="Modelo")
        self.modelo_label.grid(row=5, column=0, sticky='W', padx=5, pady=2)
        
        # Carga Label
        self.carga_label = ttk.Label(self.master, text="Carga:")
        self.carga_label.grid(row=2, column=0, sticky='W', padx=5, pady=2)


        self.monto_label = ttk.Label(self.master, text="Monto:")
        self.monto_label.grid(row=4, column=0, sticky='W', padx=5, pady=2)
        # Tipo Label
        self.tipo_label = ttk.Label(self.master, text="Tipo:")
        self.tipo_label.grid(row=3, column=0, sticky='W', padx=5, pady=2)

        # pedido Label
        self.pedido_label = ttk.Label(self.master, text="Litros cargados:")
        self.pedido_label.grid(row=6, column=0, sticky='W', padx=5, pady=2)

        # 'Get Placa' button
        self.get_placa_button = ttk.Button(self.master, text="Obtener Placa", command=self.iniciar_modelo)
        self.get_placa_button.grid(row=0, column=2, padx=5, pady=5)

        # Validate button
        self.validate_button = ttk.Button(self.master, text="Validar Placa", command=self.validate)
        self.validate_button.grid(row=1, column=0, columnspan=3, padx=5, pady=5)

        self.pedido_entry.delete(0, 'end')
        self.pedido_entry.insert(0,0) 
        # Dispat button
        self.dispatch_button = ttk.Button(self.master, text="Completar pedido", command=self.dispatch)
        self.dispatch_button.grid(row=5, column=2, columnspan=3, padx=5, pady=10)

    def iniciar_modelo(self):
        try:
            # Iniciar el temporizador para medir el tiempo de ejecución del modelo
            start_time = time.time()
            
            # Variable global para almacenar las placas detectadas
            global placas_detectadas
            
            # Iniciar la captura de video desde la cámara web
            cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
            
            # Configurar el tamaño del marco de captura
            cap.set(3, 640)  # Ancho del frame
            cap.set(4, 480)  # Altura del frame

            # Inicialización de variables
            total_frames = 0  # Contador de frames procesados
            frames_with_plates_count = 0  # Contador de frames con placas detectadas

            placas_detectadas = []  # Lista para almacenar las placas detectadas

            # Bucle para capturar continuamente y procesar los frames
            while True:
                # Leer el siguiente frame
                ret, frame = cap.read()
                if not ret:
                    break

                # Incrementar el contador de frames
                total_frames += 1

                # Convertir el frame a escala de grises
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

                # Detectar placas en el frame
                plates = plate_cascade.detectMultiScale(gray, scaleFactor=1.14, minNeighbors=7, minSize=(20, 20))

                # Si se detecta al menos una placa en el frame
                if len(plates) > 0:
                    frames_with_plates_count += 1

                    # Iterar sobre las placas detectadas en el frame
                    for (x, y, w, h) in plates:
                        # Dibujar un rectángulo alrededor de la placa detectada
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

                        # Recortar la región de interés (ROI) que contiene la placa
                        plate_roi = frame[y:y + h, x:x + w]
                        plate_gray = cv2.cvtColor(plate_roi, cv2.COLOR_BGR2GRAY)
                        
                        # Aplicar umbralización para mejorar la detección de caracteres
                        _, plate_binary = cv2.threshold(plate_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                        
                        # Utilizar Tesseract OCR para reconocer los caracteres de la placa
                        text = pytesseract.image_to_string(
                            plate_binary,
                            config='--psm 6 -c tessedit_char_whitelist=ABCDEFGHJKLMNPRSTUVWXYZ0123456789'
                        ).strip()
                        text = text.replace(" ", "")
                        
                        # Si se reconocen más de 6 caracteres (asumiendo una placa válida)
                        if len(text) > 6:
                            placas_detectadas.append(text)

                # Mostrar el frame con las placas detectadas
                cv2.imshow('Plates Detected', frame)
                
                # Verificar si el tiempo de ejecución ha excedido 8 segundos
                elapsed_time = time.time() - start_time
                if elapsed_time >= 8:
                    break
                    
                # Verificar si se presiona la tecla 'q' para salir del bucle
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            # Liberar la captura de video y cerrar las ventanas abiertas
            cap.release()
            cv2.destroyAllWindows()
            
            # Mostrar la última placa detectada en la interfaz
            self.placa_modelo()
            
            # Si se detecta al menos una placa, proceder a la validación
            if len(placas_detectadas) >= 1:
                self.validate()
                
        except Exception as e:
            # Manejo de excepciones en caso de error durante la ejecución del modelo
            
            # Mensaje de error mostrado en la interfaz
            error_message = f"Error al iniciar el modelo: {str('NO SE REALIZO UNA DETECCION VALIDA')}"
            error_label = tk.Label(self.master, text=error_message, fg="red")
            error_label.grid(row=7, column=0, columnspan=3, pady=10)

    def placa_modelo(self):
        """Funcion para colocar el ultimo valor detectado por el modelo en la interfaz"""
        quote = placas_detectadas[-1]
        self.placa_entry.insert(0, quote)

    def validate(self):
        # Función de validación ficticia
        try:
            # Obtiene la placa del campo de entrada y crea un diccionario de datos con la placa
            data = {"Placa": str(self.placa_entry.get())}
            # Realiza una solicitud GET al servidor para obtener información del automóvil utilizando la placa
            r = requests.get(str(ip_aws+'/car/readCar'), json=data)
            # Carga los datos del automóvil obtenidos en formato JSON
            car_data = json.loads(r.text)
            try:
                # Elimina y luego inserta el modelo del automóvil en un campo de entrada en la interfaz gráfica
                self.modelo_entry.delete(0, 'end')
                self.modelo_entry.insert(0, car_data["Modelo"])
                # Realiza una solicitud GET al servidor para obtener el historial de carga del usuario del automóvil
                r2 = requests.get(str(ip_aws+'/history/currentCharge/'+str(car_data["ID_usuario"])))
                # Carga los datos del historial de carga obtenidos en formato JSON
                history_data = json.loads(r2.text)
                try:
                    # Elimina y luego inserta el monto de la carga en un campo de entrada en la interfaz gráfica
                    self.monto_entry.delete(0, 'end')
                    self.monto_entry.insert(0, history_data["Monto"])
                    # Elimina y luego inserta la carga en un campo de entrada en la interfaz gráfica
                    self.carga_entry.delete(0, 'end')
                    self.carga_entry.insert(0, history_data["Carga"])
                    # Elimina y luego inserta el tipo de gas en un campo de entrada en la interfaz gráfica
                    self.tipo_entry.delete(0, 'end')
                    self.tipo_entry.insert(0, history_data["TipoGas"])
                    # Elimina y luego inserta la carga en un campo de entrada en la interfaz gráfica
                    self.pedido_entry.delete(0, 'end')
                    self.pedido_entry.insert(0, history_data["Carga"])
                except Exception as e:
                    # Muestra un mensaje de error si no hay pedido pendiente para esa placa o algún error ocurre
                    messagebox.showinfo("ERROR", "SIN PEDIDO PENDIENTE, VERIFIQUE LA PLACA O SU ORDEN")
            except Exception as e:
                # Muestra un mensaje de error si no hay registro en el sistema para esa placa o algún error ocurre
                messagebox.showinfo("ERROR", "SIN REGISTRO EN SISTEMA, VERIFIQUE LA PLACA O SU ORDEN")
        except Exception as e:
            # Muestra un mensaje de error si no hay sistema o hay un error en la conexión al servidor
            messagebox.showinfo("ERROR", "NO HAY SISTEMA")

    def dispatch(self):
        try:
            # Obtiene la placa del campo de entrada y crea un diccionario de datos con la placa
            data = {"Placa": str(self.placa_entry.get())}
            # Realiza una solicitud GET al servidor para obtener información del automóvil utilizando la placa
            r = requests.get(str(ip_aws+'/car/readCar'), json=data)
            # Carga los datos del automóvil obtenidos en formato JSON
            car_data = json.loads(r.text)
            try:
                # Realiza una solicitud GET al servidor para obtener el historial de carga del usuario del automóvil
                r2 = requests.get(str(ip_aws+'/history/currentCharge/'+str(car_data["ID_usuario"])))
                # Carga los datos del historial de carga obtenidos en formato JSON
                history_data = json.loads(r2.text)
                try:
                    # Actualiza el estado del historial de carga a "Completado"
                    data2 = {"ID_historial": history_data["ID_historial"], "Estatus": "Completado"}
                    r3 = requests.get(str(ip_aws+'/history/updateStatus'), json=data2)
                    if self.pedido_entry.get() != history_data["Carga"]:
                        # Calcula el costo por litro y el efectivo generado por la carga completa
                        costo_litro = history_data["Monto"] / history_data["Carga"]
                        carga_total = history_data["Carga"] - float(self.pedido_entry.get())
                        cash = carga_total * costo_litro
                        # Obtiene y actualiza el efectivo total del usuario (cashback)
                        data3 = {"ID_usuario": car_data["ID_usuario"]}
                        r4 = requests.get(str(ip_aws+'/user/readUser'), json=data3)
                        user_data = json.loads(r4.text)
                        cash_total = user_data["Cashback"] + cash
                        data4 = {"ID_usuario": car_data["ID_usuario"], "Cashback": cash_total}
                        r5 = requests.get(str(ip_aws+'/user/updateCashback'), json=data4)
                    # Muestra un mensaje de éxito al completar la orden
                    messagebox.showinfo("ORDEN COMPLETADA :)", "Orden completada")
                except Exception as e:
                    # Muestra un mensaje de error si no hay pedido pendiente para esa placa o algún error ocurre
                    messagebox.showinfo("ERROR", "SIN PEDIDO PENDIENTE, VERIFIQUE LA PLACA O SU ORDEN")
            except Exception as e:
                # Muestra un mensaje de error si no hay registro en el sistema para esa placa o algún error ocurre
                messagebox.showinfo("ERROR", "SIN REGISTRO EN SISTEMA, VERIFIQUE LA PLACA O SU ORDEN")
        except Exception as e:
            # Muestra un mensaje de error si no hay sistema o hay un error en la conexión al servidor
            messagebox.showinfo("ERROR", "NO HAY SISTEMA")
def leer_sensor_y_ejecutar_modelo():
    try:
        # Establece la conexión con el dispositivo Arduino a través del puerto COM3 con una velocidad de 9600 baudios
        arduino = serial.Serial("COM3", 9600)
        time.sleep(2)  # Espera 2 segundos para asegurar la estabilización de la conexión

        while True:
            # Lee los datos del sensor conectado a Arduino a través del puerto serial
            datos = arduino.readline()
            # Convierte los datos leídos a un entero decodificando la información en formato UTF-8
            datos_int = int(datos.decode('utf-8'))

            # Verifica si los datos leídos son menores o iguales a 6
            if datos_int <= 6:
                # Si los datos indican una distancia menor o igual a 6, se ejecuta la función iniciar_modelo
                print("Distancia menor o igual a 6, ejecutando la función iniciar_modelo...")
                app.iniciar_modelo()

    except serial.SerialException as e:
        # Maneja excepciones relacionadas con la conexión al dispositivo Arduino
        error_message = f"Error con Arduino: {str(e)}"
        print(error_message)
        # Muestra un mensaje de error relacionado con Arduino en la interfaz gráfica
        messagebox.showinfo("Error con Arduino", error_message)

    except Exception as e:
        # Maneja excepciones generales o desconocidas
        error_message = f"Error desconocido: {str(e)}"
        print(error_message)
        # Muestra un mensaje de error desconocido en la interfaz gráfica
        messagebox.showinfo("Error Desconocido", error_message)

if __name__ == "__main__":
    # Inicializa la ventana principal de la aplicación con la detección de placas
    root = tk.Tk()
    root.state('zoomed')  # Configura la ventana para que se maximice al iniciar
    root.title("Detección de Placas")

    # Crea una instancia de la aplicación
    app = Application(master=root)

    # Inicia un hilo para leer el sensor y ejecutar el modelo
    sensor_thread = threading.Thread(target=leer_sensor_y_ejecutar_modelo)
    sensor_thread.start()

    # Intenta abrir la aplicación en el segundo monitor si está disponible
    monitors = get_monitors()
    if len(monitors) > 1:
        second_monitor = monitors[1]  # Obtiene el segundo monitor
        # Ubica la ventana en el segundo monitor con su tamaño completo
        root.geometry(f"{root.winfo_screenwidth()}x{root.winfo_screenheight()}+{second_monitor.x}+{second_monitor.y}")
    
    root.mainloop()  # Inicia el bucle principal de la interfaz gráfica


Distancia menor o igual a 6, ejecutando la función iniciar_modelo...
Distancia menor o igual a 6, ejecutando la función iniciar_modelo...
Distancia menor o igual a 6, ejecutando la función iniciar_modelo...
Distancia menor o igual a 6, ejecutando la función iniciar_modelo...


Error desconocido: main thread is not in main loop
